In [1]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [2]:
#!python3 -m pip install --upgrade pip
#!pip install pydot
#!apt-get install -y graphviz

In [3]:
import os,shutil,random
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
from sklearn.model_selection import train_test_split
import datetime as dt
import numpy as np
#================== initialization ==================
currentTM=dt.datetime.now().strftime("%Y-%m-%dT%H%M%S")
PROJECT = "testVAEModel"
LATENT_DIM = 128
VAE_LR = 1e-4
EPOCHS = 1
BATCH_SIZE = 32
PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
MODEL_PATH = "./models/"
H5_PATH = './data/OptTrainData/'
DISPLAY = False
WANDB_LOG = False
RESUME = False

2024-07-04 15:02:55.880997: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#741d760b304d0be5b18d4ee9682f77156e6967b5
import wandb
from wandb.keras import WandbCallback
CONFIG = {    "latent_dim":LATENT_DIM,
                  "learning_rate": VAE_LR,
                  "epochs": EPOCHS,
                  "batch_size": BATCH_SIZE,
                  "architecture": "VAE",
                  "dataset": "OptionsChaine",
                  "encoder_dense_units":[512,256],
                  "encoder_dropout_rate":0.2,
                  "decoder_dense_units":[256, 512],
                  "decoder_dropout_rate":0.2,
               }
    
if WANDB_LOG :
    wandb.login()
    run = wandb.init(project=PROJECT, name=currentTM, config=CONFIG)

In [5]:
#Example

# from IPython.display import clear_output,display, HTML
# import numpy as np
# #load scaler
# scaler = MinMaxScaler()
# PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
# random.shuffle(PartitionDate)
# scaler = joblib.load(SCALER_PATH)


# for i,partdate in enumerate(PartitionDate) :
#     df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
#                                  , filters=[('PartitionDate', '=', partdate)]
#                                 )
#     df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
#     df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
#     DATA  = np.empty((0,) + (20,9) ) 
#     for opt_id in np.unique( df[["OPTIONS_ID"]].values):
#         df_filter  = df[df["OPTIONS_ID"]==opt_id]
#         if len(df_filter) == 20:
#             DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
#         else:
#             #print( len(df_filter) )
#             #display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
#             pass
            
#     ## Save the NumPy array to an HDF5 file
#     # with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
#     #     dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

#     print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [6]:
# # Save the NumPy array to an HDF5 file
# with h5py.File(H5_PATH, 'w') as f:
#     #dset = f.create_dataset('dataset', data=DATA, chunks=True, compression='gzip')
#     #test
#     dset = f.create_dataset('dataset', data=DATA, compression='gzip')

In [7]:
#=====================================================================

In [8]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= (20,3), 
            dense_units = CONFIG["encoder_dense_units"], 
            dropout_rate= CONFIG["encoder_dropout_rate"]
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= (20,1),
            dense_units = CONFIG["decoder_dense_units"],
            dropout_rate= CONFIG["decoder_dropout_rate"]
           )
)

def dummy_loss(y_true, y_pred):
    return 0.0
    
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer )#, loss=dummy_loss)

In [9]:
################## show model ######################
if DISPLAY :
    from tensorflow.keras.utils import model_to_dot
    from IPython.display import SVG, display
    
    def display_model(model, width=1024, height=512):
        dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
        svg_data = dot.create(prog='dot', format='svg').decode("utf-8")
        svg_html = f'<div style="width:{width}px;height:{height}px;">{svg_data}</div>'
        display(HTML(svg_html))

In [10]:
## Example usage:
## Display the encoder model with reduced size
if DISPLAY :
    display_model(model.encoder, width=1024, height=512)

In [11]:
if DISPLAY :
    display_model(model.decoder, width=2500, height=512)

In [12]:
#================== loadmodel ====================

In [13]:
from tensorflow.keras.models import load_model
model_path = MODEL_PATH+f'{PROJECT}'
if not RESUME :
    if os.path.exists(model_path) :
        shutil.rmtree(model_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
else:
    model.encoder = load_model(model_path+'/'+f'encoder.keras') 
    model.decoder = load_model(model_path+'/'+f'decoder.keras') 

In [14]:
#================== train model ==================
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)

STOP_MODEL = False
for partdate in PartitionDate[:] :
    #nan problem
    #partdate = '2022-05'
    #normal 
    #partdate = '2011-12'
    
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
        X = DATA[:, :, :3]  # เลือกข้อมูลแถวแรกถึงแถวที่ 3 สำหรับ X
        Y = DATA[:, :, 3:]  # เลือกข้อมูลแถวที่ 3 เป็นต้นไปสำหรับ Y
        x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
        if len(X) :
            random.shuffle(PartitionDate)
            tf.keras.backend.clear_session() 
            history = model.fit(x_train , y_train, epochs=5, batch_size=BATCH_SIZE, validation_data=(x_val, y_val) )
            if  np.isnan(  np.average( history.history['kl_loss'] )  ) or np.isnan(  np.average( history.history['val_kl_loss'] )  ):
                STOP_MODEL = True 
                print(x_train)
                print("---")
                print(x_val)
                print("=============")
    if WANDB_LOG :
        wandb.log({
            "kl_loss": np.average(  history.history['kl_loss'] )
            ,"total_loss":np.average(  history.history['total_loss'] )
            #,"vol_loss":np.average(  history.history['vol_loss'] )

            ,'val_kl_loss':np.average(  history.history['val_kl_loss'] )
            ,'val_total_loss':np.average(  history.history['val_total_loss'] )
            #, 'val_vol_loss':np.average(  history.history['val_vol_loss'] )
        }, commit=True)
        
        # wandb.log({
        #     "kl_loss": np.average(  history.history['kl_loss'] )
        #     ,"total_loss":np.average(  history.history['total_loss'] )
        #     ,"loss":np.average(  history.history['loss'] )
        #     ,"optVal_loss":np.average(  history.history['optVal_loss'] )
        #     ,"vol_loss":np.average(  history.history['vol_loss'] )
        # }, commit=True)

    if STOP_MODEL :
        break
    
            
    model.encoder.save(model_path+f'/'+f'encoder.keras') 
    model.decoder.save(model_path+f'/'+f'decoder.keras') 
if WANDB_LOG : wandb.finish()

Epoch 1/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - kl_loss: 0.0229 - total_loss: 0.3780 - val_kl_loss: 6.2879e-04 - val_total_loss: 0.0265
Epoch 2/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - kl_loss: 0.0027 - total_loss: 0.0617 - val_kl_loss: 2.7977e-04 - val_total_loss: 0.0105
Epoch 3/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - kl_loss: 9.4000e-04 - total_loss: 0.0257 - val_kl_loss: 1.6374e-04 - val_total_loss: 0.0072
Epoch 4/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - kl_loss: 4.8064e-04 - total_loss: 0.0171 - val_kl_loss: 9.2700e-05 - val_total_loss: 0.0068
Epoch 5/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - kl_loss: 2.6745e-04 - total_loss: 0.0113 - val_kl_loss: 5.1607e-05 - val_total_loss: 0.0063
Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - kl_loss: 1.7262e-04 - total_loss: 0.0090 - val_kl_loss: 2.9534e-05 - val_total_loss: 0.0050
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 1.1430e-04 - total_loss: 0.0076 - val_kl_loss: 1.9562e-05 - val_total_loss: 0.0049
Epoc

2024-07-04 15:04:05.889575: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: c_bid contains NaNs or Infs ----- Tensor("strided_slice_2:0", shape=(None, 20), dtype=float32) : Tensor had NaN values
	 [[{{function_node __inference_one_step_on_data_4548}}{{node CheckNumerics}}]]


InvalidArgumentError: Graph execution error:

Detected at node CheckNumerics defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipykerne

In [ ]:
`====================================================

In [ ]:
print(x_train)


In [27]:
a=tf.expand_dims(y_val[:, :, 0], axis=-1).numpy()

In [49]:
np.sum(Y)

nan

In [47]:
for i in a:
    if np.isnan( np.sum(i) ) :
        print(i)
        print('-------')
        break

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
-------


In [46]:
np.sum(i)

0.007469971563176436

In [ ]:
print(x_val)
print("---")
print(y_val)
print("=============")

In [15]:
Y=y_val
for i in range ( len( np.transpose( Y[0] ) ) ):
    col = np.transpose( Y[i] ) 
    for c in range(len(col)):
        a = np.transpose( Y[i] )[c]
        if np.sum( a ) == 0 :
            print(f"0 - i:{i},c:{c}")
        if np.isnan( np.sum( a ) ) :
            print(f"nan - i:{i},c:{c}")

0 - i:4,c:2
0 - i:4,c:5
0 - i:5,c:2


In [16]:
 np.transpose( y_val[4] )[2]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [ ]:
import pandas as pd

In [ ]:

import datetime as dt
{
    "data": {
        "MANDT": "WA",
        "BUKRS": "2011500",
        "LIFNR1": "P10005418",
        "LIFNR2": "P0754010",
        "CURRENCY": "THB",
        "AMOUNT": 10000.00,
        "VNAME": "ศูนย์กระจายสินค้า-RDC.ลำปาง"
    },
    "eventName": "PRODUCE",
    "messageDate": dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
    "messageKey": "161143",
    "messageVersion": "1.7.0"
}

In [ ]:
import datetime as dt


In [ ]:
DATA = [{'WA': '2011500P10005418 P0754010  THB      10000.00 ศูนย์กระจายสินค้า-RDC.ลำปาง'}, {'WA': '2011500P10005418 P10005418 THB      20000.00 จำลอง แก้วโพธิ์คา'}, {'WA': '2011500P1001216  P1001216  THB      50000.00 กฤษณา เงินสดย่อย'}, {'WA': '2011500P11002595 P11002595 THB      20000.00 ธาริน  พินเที่ยง'}, {'WA': '2011500P11004112 P11004112 THB       2000.00 สันติ  กาบก้วง'}, {'WA': '2011500P11004119 P11004119 THB       3000.00 ยุทธนา  แก้วพูลศรี'}, {'WA': '2011500P11004120 P11004120 THB       3000.00 อำนาจ  พึ่งเจียม'}, {'WA': '2011500P11004134 P11004134 THB      20000.00 บรรลือศักดิ์  สาครพันธรักษ์'}, {'WA': '2011900P70092728 P70092728 THB      50000.00 กิตติอาภา อ่างบุญพงษ์'}, {'WA': '2011900P91020000 P91020000 THB      50000.00 วิไลภรณ์ ภัทรพงศาวณิชย์'}, {'WA': '20119D0P70092728 P70092728 THB      50000.00 กิตติอาภา อ่างบุญพงษ์'}, {'WA': '20119D0P91020000 P91020000 THB      50000.00 วิไลภรณ์ ภัทรพงศาวณิชย์'}, {'WA': '2012200P1001742  P1001742  THB      30000.00 วรัญญา เงินสดย่อย'}, {'WA': '2012200P11000373 P11000373 THB      22000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2012200P11001443 P11001443 THB      40000.00 วรกานท์  สมบูรณ์บัติ'}, {'WA': '2012200P11007055 P11007055 THB      50000.00 บุษบง  จันทร์สว่าง'}, {'WA': '2012200P11033138 P11033138 THB      80000.00 แสงโสม Petty Cash'}, {'WA': '2012200P11033139 P11033139 THB      50000.00 พรทิพย์ ปิยะอรุณ Test'}, {'WA': '2012200P70082951 P70082951 THB       9000.00 ชานน บุญญาศิริรัตน์'}, {'WA': '2012200P700927288P700927288THB      50000.00 petty cash kittiarpa'}, {'WA': '2012300P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2012300P11005262 P11005262 THB      50000.00 มุกดา  จันทร์เพ็ญ'}, {'WA': '2012400P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2012400P11006877 P11006877 THB      50000.00 จุไรรัตน์  จันทร์แดง'}, {'WA': '2012500P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2012500P11006990 P11006990 THB      50000.00 จิราพร วงศ์ษา'}, {'WA': '2012600P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2012600P11007055 P11007055 THB      50000.00 บุษบง  จันทร์สว่าง'}, {'WA': '2012700P11000373 P11000373 THB      22000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2012700P11008749 P11008749 THB      50000.00 สุพัตรา  วสุลิปิกร'}, {'WA': '2012800P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2012800P11020046 P11020046 THB      50000.00 สุกัญญา เนืองภา'}, {'WA': '2012900P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013000P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013000P11007240 P11007240 THB      50000.00 วัชราภรณ์  เนตรนี'}, {'WA': '2013100P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013100P11012237 P11012237 THB      50000.00 อาภากร ผาทอง'}, {'WA': '2013200P11000373 P11000373 THB     462000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013200P11009232 P11012237 THB      50000.00 อาภากร ผาทอง'}, {'WA': '2013200P11017806 P11017806 THB      30000.00 จารุกร จันทร์แรม'}, {'WA': '2013300P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013300P11009394 P11009394 THB      50000.00 วันเพ็ญ  ยุดรัมย์'}, {'WA': '2013400P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013400P11013688 P11013688 THB      50000.00 น้ำอ้อย   สุภิรมย์'}, {'WA': '2013400P11017806 P11017806 THB      50000.00 จารุกร จันทร์แรม'}, {'WA': '2013500P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013500P11000998 P11000998 THB      50000.00 สรัญญา  ด่านวงศ์ทอง(เงินสดย่อย-รง.)'}, {'WA': '2013600P11000373 P11000373 THB       7000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013600P11031653 P11031653 THB      50000.00 ฉัตรกมล สุวรรณศิริ'}, {'WA': '2013700P11000373 P11000373 THB       4000.00 ศิริวรรณ มณีสุขโรจน์'}, {'WA': '2013800P11009394 P11009394 THB      50000.00 วันเพ็ญ  ยุดรัมย์'}, {'WA': '2014000P11001818 P11001818 THB      35000.00 เอกพจน์  รัตนเศวตสรรค์'}, {'WA': '2014000P11001925 P11001925 THB      35000.00 ทองสูน คงความเจริญ'}, {'WA': '2014000P11001996 P11001996 THB      35000.00 กฤษณะ  อ่อนอรุณ'}, {'WA': '2014000P11002095 P11002095 THB      35000.00 สัณฑน  แสงมโนทัย'}, {'WA': '2014000P11029172 P11029172 THB      35000.00 ชาญวิทย์ บุญยัง'}, {'WA': '2014000P11042800 P11042800 THB      35000.00 อภิชา ศรีสง่า'}, {'WA': '2014000P11042900 P11042900 THB      35000.00 วสันต์ พงษ์มาตรสุวร'}, {'WA': '2014000P70092728 P11033138 THB      30000.00 แสงโสม Petty Cash'}, {'WA': '2014200P1001742  P1001742  THB      50000.00 วรัญญา เงินสดย่อย'}, {'WA': '2014200P11014200 P11014200 THB      50000.00 สมฤดี วิเศษชูชาติ'}, {'WA': '2014200P11025891 P11025891 THB       5000.00 ฐิติรัตน์  ศรีจันทร์'}, {'WA': '2014300P11000236 P11000236 THB      20000.00 นายสมภพ  กองทรัพย์และนางสาววรรณา เก'}, {'WA': '2014300P11033811 P11033811 THB      50000.00 Waraporn Paenkaew'}, {'WA': '2014300P70077557 P11033811 THB      20000.00 Waraporn Paenkaew'}, {'WA': '2014300P70077557 P70077557 THB      30000.00 Thanutchaporn'}, {'WA': '2014500P11004104 P11004104 THB      20000.00 จุฑามาศ ขวัญคำ'}, {'WA': '2014800P11005412 P11005406 THB      40000.00 กชวรรณ พูลพงษ์'}, {'WA': '2014800P11005412 P11005412 THB      45000.00 มณฑา  ก้วยไข่มุก'}, {'WA': '2014800P11005412 P11022466 THB      25000.00 ขณิษฐา แซ่โต๋ว'}, {'WA': '2014800P11005412 P11030049 THB       3000.00 สุธาทิพย์ ดวงจันทร์'}, {'WA': '2014800P11005412 P11030937 THB       3000.00 สุพนิดา โสตรโยม'}, {'WA': '2014800P11005412 P11031191 THB       3000.00 วนิดา สาพิลาศ'}, {'WA': '2014800P11005412 P11032161 THB       3000.00 ศิวาพร เมืองศรี'}, {'WA': '2014800P11005412 P11032785 THB      31000.00 จิตรอารีย์ บัวเรียน'}, {'WA': '2014800P11005412 P11033290 THB       3000.00 ปทิตา สิงห์บล'}, {'WA': '2014800P11005412 P11034855 THB       3000.00 รัตนาพร มาโยธา'}, {'WA': '2014800P11005412 P70005334 THB       3000.00 สุปรียา ปราบไพรี'}, {'WA': '2014900P11004935 P11004935 THB      50000.00 กฤติภรณ์  คมขำ'}, {'WA': '2014900P11004935 P11005119 THB       5000.00 สมบัติ ใยพูล'}, {'WA': '2014900P11004935 P11005139 THB       5000.00 กิตติเชษฐ์ ใจบุญ'}, {'WA': '2014900P11004935 P11005218 THB       5000.00 คัคนางค์ ประวัติศิริ'}, {'WA': '2014900P11004935 P11005220 THB       5000.00 ถนอมจิต เฉลิมพักตร์'}, {'WA': '2014900P11004935 P11005341 THB       5000.00 กฤษณา สาเกตุ'}, {'WA': '2014900P11004935 P11005403 THB       5000.00 นิตยา เทียนสิริ'}, {'WA': '2014900P11004935 P11005453 THB       5000.00 นิตยา เจริญโสภา'}, {'WA': '2014900P11004935 P11005514 THB       5000.00 กฤษณา กาญจนระพีพรรณ'}, {'WA': '2014900P11004935 P11005571 THB       5000.00 อัจฉรา แย้มบุญชู'}, {'WA': '2014900P11004935 P11005606 THB       5000.00 อัจฉรี แย้มบุญชู'}, {'WA': '2014900P11004935 P11005613 THB       5000.00 ศรีประภา วัฒนเกษมวงศ์'}, {'WA': '2014900P11004935 P11005661 THB       5000.00 ชุติมา สวัสดีมีโชค'}, {'WA': '2014900P11004935 P11005887 THB       5000.00 นราศรี แจ้งศรี'}, {'WA': '2014900P11004935 P11005936 THB       5000.00 สุปราณี หร่ายเกษม'}, {'WA': '2014900P11004935 P11006166 THB       5000.00 มนัสวินี แก้วกำเนิด'}, {'WA': '2014900P11004935 P11006189 THB       5000.00 ผุสดี ทองเสนอ'}, {'WA': '2014900P11004935 P11006203 THB       5000.00 ลลิตา ปลูกสมบัติ'}, {'WA': '2014900P11004935 P11013058 THB       5000.00 มะลิวรรณ์ สิงห์ทอง'}, {'WA': '2014900P11004935 P11013108 THB       5000.00 สุธาสินี ชูติวาส'}, {'WA': '2014900P11004935 P11014219 THB       5000.00 รัศมี สุตมาตร์'}, {'WA': '2014900P11004935 P11020236 THB       5000.00 จุฬารัตน์ องคเชษฐ์'}, {'WA': '2014900P11004935 P11024842 THB       5000.00 นิภารัตน์ สะปู'}, {'WA': '2014900P11004935 P11025250 THB       5000.00 เดียงสา เตชะบุรัมย์'}, {'WA': '2014900P11004935 P11025251 THB       5000.00 สุปราณี วงศาโรจน์'}, {'WA': '2014900P11004935 P11025765 THB      10000.00 นภาพรรณ สนิทเอื้อ'}, {'WA': '2014900P11004935 P11027425 THB      10000.00 ธนภร โยธาธนพาณิชย์'}, {'WA': '2014900P11004935 P11027626 THB       5000.00 ประภาทิพย์ จตุพรมงคลชัย'}, {'WA': '2014900P11004935 P11028051 THB       5000.00 อัญชลีพร  เกตุทัต'}, {'WA': '2014900P11004935 P11028816 THB       5000.00 ศิริกุล  หล่าธรรม'}, {'WA': '2014900P11004935 P21008598 THB       5000.00 เสาวลักษณ์  นามพุทธา'}, {'WA': '2014900P11004935 P21031931 THB      10000.00 นุ่นนภา ขมิ้นทอง'}, {'WA': '2014900P11004935 P21048306 THB       5000.00 มยุรา  ทองปา'}, {'WA': '2014900P11009695 P11005505 THB      10000.00 ดาหวัน อรุณคีรีวัฒน์'}, {'WA': '2014900P11009695 P11007814 THB       5000.00 ขันทอง ห้วยกรุด'}, {'WA': '2014900P11009695 P11008177 THB       5000.00 อรัญญา สายสวาท'}, {'WA': '2014900P11009695 P11009686 THB       5000.00 กุลวัลย์ บุญธรรม'}, {'WA': '2014900P11009695 P11009695 THB      20000.00 ภรทนา วิสุทธิแพทย์'}, {'WA': '2014900P11009695 P11010292 THB       5000.00 กมลลักษณ์ เสาสุดใจ'}, {'WA': '2014900P11009695 P11010407 THB       5000.00 ศรัณญา วงศ์กัลยา'}, {'WA': '2014900P11009695 P11010470 THB       5000.00 วาสนา เฉลิมวงษ์'}, {'WA': '2014900P11009695 P11010592 THB       5000.00 ชุติมา มณีพันธ์'}, {'WA': '2014900P11009695 P11010616 THB       5000.00 นิตยา โพธิ์ผลิ'}, {'WA': '2014900P11009695 P11024243 THB       5000.00 นิลเนตร วงศ์มณี'}, {'WA': '2014900P11009695 P11030147 THB       5000.00 รัตนา บุญมา'}, {'WA': '2014900P11009695 P11030908 THB       5000.00 เพ็ญนภา ชูเลิศ'}, {'WA': '2014900P11009695 P11031527 THB       5000.00 สุดาพร บุตรจา'}, {'WA': '2014900P11009695 P11033058 THB       5000.00 เสกสรรค์ มีทองพูล'}, {'WA': '2014900P11009695 P11033769 THB       5000.00 มนฤดี เหลือวงศ์'}, {'WA': '2014900P11028578 P11028578 THB     100000.00 อุดมรักษ์ ซื่อสัตย์'}, {'WA': '2015000P11030300 P11030300 THB       5000.00 รัชนก  บุญยอด'}, {'WA': '2015000P70034664 P19E00007 THB       1000.00 EatPot Chaengwattana'}, {'WA': '2015000P70034664 P70034664 THB       2000.00 รัตติยาภรณ์  ศรีวรรณ'}, {'WA': '2015100P70034665 P70034665 THB       3000.00 ปวิชญา บุญทระกุล'}, {'WA': '2015200P70034665 P70034665 THB       3000.00 ปวิชญา บุญทระกุล'}, {'WA': '2015400P70038535 P70038535 THB      15000.00 สุนทรี ศิริมงคล'}, {'WA': '2015400P70051204 P70051204 THB      10000.00 ภัทรวดี  หนูเครือ'}, {'WA': '2015500P70034664 P70034664 THB      10000.00 รัตติยาภรณ์  ศรีวรรณ'}, {'WA': '2015500P70073483 P70073483 THB      15000.00 พรปวีณ์ มีทรัพย์ทอง'}, {'WA': '2015500P70082276 P70082276 THB      15000.00 จุฬาลักษณ์ ทองสิมา'}, {'WA': '2015600P11005308 P11005308 THB      30000.00 สมร ค้ายาดี'}, {'WA': '2015600P11005819 P11005819 THB      30000.00 สมร  ค้ายาดี'}, {'WA': '2015700P11000126 P11000126 THB      50000.00 วิภัสสรา ส.เจริญพานิช'}, {'WA': '2015700P11000126 P11000127 THB      50000.00 วิมลทิพย์ เจริญกุล'}, {'WA': '2015700P11000128 P11000128 THB     150000.00 ท่องเที่ยว เมืองไทย'}, {'WA': '2015800P11010957 P11008939 THB      50000.00 แจ่มจันทร์ พลศรี'}, {'WA': '2015800P11010957 P11008963 THB       5000.00 หอมไกร ทาทะสุทธิ์'}, {'WA': '2015800P11010957 P11010170 THB      40000.00 พิกุล สำราญวงศ์'}, {'WA': '2015800P11010957 P11010171 THB      40000.00 วีณา ชัยประโคน'}, {'WA': '2015800P11010957 P11010957 THB      80000.00 สุคันธรส จรัสฤกษ์วงศ์'}, {'WA': '2015800P11010957 P11011925 THB       5000.00 จันจิรา แสงจันทร์'}, {'WA': '2015800P11010957 P11026302 THB       5000.00 อมรรัตน์ กล้ามกระโทก'}, {'WA': '2015800P11010957 P11028352 THB       5000.00 ศิวาภรณ์ อัพภันดร'}, {'WA': '2015800P11027011 P11027011 THB      50000.00 ฉัตรสุรางค์ เรืองศิลป์'}, {'WA': '2015900P11010156 P11008290 THB       5000.00 นันทนา สังข์ทอง'}, {'WA': '2015900P11010156 P11010150 THB      30000.00 กนกวรรณ มีจันทร์โท'}, {'WA': '2015900P11010156 P11010153 THB      50000.00 ลักขณา อยู่วรรณ'}, {'WA': '2015900P11010156 P11010156 THB      60000.00 กอบสิน สุภาวะ'}, {'WA': '2015900P11010156 P11010160 THB      40000.00 คณานิช แสนมงคล'}, {'WA': '2015900P11010156 P11012313 THB       5000.00 ลลิตา บุญณวี'}, {'WA': '2015900P11010156 P11012619 THB       3000.00 กรรณิการ์ ทองน้อย'}, {'WA': '2015900P11010156 P11012878 THB       3000.00 วิมล เมืองด่าน'}, {'WA': '2015900P11010156 P11019544 THB       3000.00 ศิริรัตน์ บุญมา'}, {'WA': '2015900P11010156 P11024876 THB       3000.00 จันทร์เพ็ญ ก้อนเกียรติวงศ์'}, {'WA': '2015900P11010156 P11028146 THB       5000.00 นฤมล เรืองสุกใส'}, {'WA': '2015900P11010156 P11032768 THB       3000.00 เมธปิยา ไชยยืน'}, {'WA': '2015900P11010156 P21024881 THB       2000.00 ลำดวน ใจมั่น'}, {'WA': '2015900P11030173 P11000126 THB      10000.00 วิภัสสรา ส.เจริญพานิช'}, {'WA': '2015900P11030173 P11030173 THB      40000.00 พัชรนันต์ คร้ามไพบูลย์'}, {'WA': '2015900P70039286 P70025355 THB      30000.00 วาสนา ยุทธนาโยธิน'}, {'WA': '2015900P70039286 P70039286 THB      70000.00 กุลิสรา มั่งสกุล'}, {'WA': '2015900P70042000 P70042000 THB      30000.00 สุกัญญา แสงศิริ'}, {'WA': '2015900P70042000 P70042001 THB      20000.00 อรุณทอง ศรีพรรณ'}, {'WA': '2016000P1001216  P1001216  THB      40000.00 กฤษณา เงินสดย่อย'}, {'WA': '2016000P11011357 P11006117 THB      20000.00 สุกัญญา ฐิติวัฒนา'}, {'WA': '2016000P11011357 P11010148 THB      30000.00 เสาวลักษณ์ สินภูธร'}, {'WA': '2016000P11011357 P11010154 THB      15000.00 วรรณพร พลมาศ'}, {'WA': '2016000P11011357 P11010364 THB       3000.00 หทัยรัตน์ อ่อนจันทร์สกุล'}, {'WA': '2016000P11011357 P11010518 THB          0.00 รำไพวัลย์ นาเสงี่ยม'}, {'WA': '2016000P11011357 P11010678 THB       1000.00 สุรีรัตน์ แสงกล้า'}, {'WA': '2016000P11011357 P11010683 THB       3000.00 พันธ์นิภา พุ่มไหม'}, {'WA': '2016000P11011357 P11010771 THB       2000.00 ไพรัตน์  มั่นทอง'}, {'WA': '2016000P11011357 P11011184 THB      20000.00 วิมล มูลเดช'}, {'WA': '2016000P11011357 P11011357 THB      40000.00 นิลุบล ตั้นพันธ์'}, {'WA': '2016000P11011357 P11011399 THB       2000.00 นันทนา ลี่เส็ง'}, {'WA': '2016000P11011357 P11029297 THB       5000.00 อำพัน เมืองข้องน้อย'}, {'WA': '2016100P11009875 P11009736 THB      35000.00 สุรีย์ ยศมูรธา'}, {'WA': '2016100P11009875 P11009843 THB      35000.00 โสพล อ่อนแก้ว'}, {'WA': '2016100P11009875 P11009875 THB      40000.00 ยุวพันธ์ สุขสมพร'}, {'WA': '2016100P11009875 P11009906 THB       1500.00 รพีพรรณ มณีรัตน์'}, {'WA': '2016100P11009875 P11009914 THB      25000.00 สุวิดา สาครินทร์'}, {'WA': '2016100P11009875 P11010149 THB      60000.00 อุบล บุญฮก'}, {'WA': '2016100P11009875 P11016251 THB       1500.00 จันจิรา เรืองประพันธ์'}, {'WA': '2016100P11009875 P11021812 THB       4000.00 ผกาทิพย์ เรืองศรี'}, {'WA': '2016100P11009875 P11028216 THB       3000.00 พรพิมล วัจนะเสนาะ'}, {'WA': '2016100P11009875 P11030230 THB       2000.00 วไลลักษณ์ หลีไมล์'}, {'WA': '2016100P11009875 P11032943 THB      10000.00 สรัลพร รัฐวิเศษ'}, {'WA': '2016200P11002371 P11002371 THB      27500.00 นุชจรินทร์ นิ้มวัฒนา'}, {'WA': '2016200P11002371 P70029883 THB      27500.00 ธัญชนก สุขบำรุง'}, {'WA': '2016200P11002706 P11002706 THB     100000.00 จารุวรรณ  ธาระเขตร์'}, {'WA': '2016200P11005218 P11002371 THB      50000.00 นุชจรินทร์ นิ้มวัฒนา'}, {'WA': '2016200P11005218 P11002456 THB      50000.00 สุภัทรา กุสินทร์เกิด'}, {'WA': '2016200P11005336 P11005336 THB      60000.00 สุภาวดี มงคลวัฒน์'}, {'WA': '2016200P11005336 P11005771 THB       3000.00 ธิติมา เทพพิพิธ'}, {'WA': '2016200P11005336 P11005810 THB       5000.00 พงษ์พินิจ ปลื้มจิต'}, {'WA': '2016200P11005336 P11006188 THB       5000.00 สุธิน ทวนประเสริฐ'}, {'WA': '2016200P11005336 P21034579 THB       3000.00 ปถมาพร พาราพิบูลย์'}, {'WA': '2016200P11006045 P11006045 THB        200.00 ทิพย์วรรณ์   ศรีกล่ำ'}, {'WA': '2016200P11006045 P11006189 THB        100.00 ผุสดี ทองเสนอ'}, {'WA': '2016200P11010572 P11010572 THB     100000.00 ชุติมณฑน์  ยิ่งยวด'}, {'WA': '2016300P11005369 P11005369 THB      30000.00 สุวัจนา  ชิตรัตน์'}, {'WA': '2016300P11010170 P11010170 THB      50000.00 พิกุล สำราญวงศ์'}, {'WA': '2016400P11010166 P11008939 THB      50000.00 แจ่มจันทร์ พลศรี'}, {'WA': '2016400P11010166 P11010166 THB      80000.00 จิราพร พิชญ์อังกูร'}, {'WA': '2016400P11010166 P11010170 THB      40000.00 พิกุล สำราญวงศ์'}, {'WA': '2016400P11010166 P11010171 THB      40000.00 วีณา ชัยประโคน'}, {'WA': '2016400P70042111 P70042111 THB      30000.00 วรรณวิศา วงค์ศรี'}, {'WA': '2016400P70042111 P70042222 THB      10000.00 พิมพ์ภรณ์ แสงเนตร'}, {'WA': '2016500P11006003 P11006003 THB      60000.00 โสภาวัลย์ ลือชัย'}, {'WA': '2016500P11006003 P11010150 THB      50000.00 กนกวรรณ มีจันทร์โท'}, {'WA': '2016500P11006003 P11010153 THB      50000.00 ลักขณา อยู่วรรณ'}, {'WA': '2016500P11006003 P11010160 THB      60000.00 คณานิช แสนมงคล'}, {'WA': '2016500P11006003 P11012313 THB       5000.00 ลลิตา บุญณวี'}, {'WA': '2016500P11006003 P11012619 THB       3000.00 กรรณิการ์ ทองน้อย'}, {'WA': '2016500P11006003 P11019544 THB       3000.00 ศิริรัตน์ บุญมา'}, {'WA': '2016500P11006003 P11024876 THB       5000.00 จันทร์เพ็ญ ก้อนเกียรติวงศ์'}, {'WA': '2016500P70042111 P11002456 THB      30000.00 สุภัทรา กุสินทร์เกิด'}, {'WA': '2016500P70042111 P70042111 THB      70000.00 วรรณวิศา วงค์ศรี'}, {'WA': '2016600P11010145 P11006117 THB      25000.00 สุกัญญา ฐิติวัฒนา'}, {'WA': '2016600P11010145 P11010145 THB      60000.00 เพลินจิตร์ จันทรทิพย์'}, {'WA': '2016600P11010145 P11010148 THB      30000.00 เสาวลักษณ์ สินภูธร'}, {'WA': '2016600P11010145 P11010154 THB      20000.00 วรรณพร พลมาศ'}, {'WA': '2016600P11010145 P11010364 THB       2000.00 หทัยรัตน์ อ่อนจันทร์สกุล'}, {'WA': '2016600P11010145 P11010518 THB          0.00 รำไพวัลย์ นาเสงี่ยม'}, {'WA': '2016600P11010145 P11010678 THB       1000.00 สุรีรัตน์ แสงกล้า'}, {'WA': '2016600P11010145 P11010683 THB       2000.00 พันธ์นิภา พุ่มไหม'}, {'WA': '2016600P11010145 P11010771 THB       1000.00 ไพรัตน์  มั่นทอง'}, {'WA': '2016600P11010145 P11011184 THB      25000.00 วิมล มูลเดช'}, {'WA': '2016600P11010145 P11011399 THB       2000.00 นันทนา ลี่เส็ง'}, {'WA': '2016600P11010145 P11029297 THB       3000.00 อำพัน เมืองข้องน้อย'}, {'WA': '2016700P1001216  P1001216  THB      50000.00 กฤษณา เงินสดย่อย'}, {'WA': '2016700P11005067 P11005067 THB      40000.00 มาลี นาควรพงศ์'}, {'WA': '2016700P11005067 P11009736 THB      35000.00 สุรีย์ ยศมูรธา'}, {'WA': '2016700P11005067 P11009843 THB      35000.00 โสพล อ่อนแก้ว'}, {'WA': '2016700P11005067 P11009906 THB       1500.00 รพีพรรณ มณีรัตน์'}, {'WA': '2016700P11005067 P11009914 THB      25000.00 สุวิดา สาครินทร์'}, {'WA': '2016700P11005067 P11010149 THB      50000.00 อุบล บุญฮก'}, {'WA': '2016700P11005067 P11021812 THB       1000.00 ผกาทิพย์ เรืองศรี'}, {'WA': '2016700P11005067 P11028216 THB       2000.00 พรพิมล วัจนะเสนาะ'}, {'WA': '2016700P11005067 P11030230 THB       1000.00 วไลลักษณ์ หลีไมล์'}, {'WA': '2016700P11005067 P11032943 THB      10000.00 สรัลพร รัฐวิเศษ'}, {'WA': '2016800P11000126 P11000126 THB     100000.00 วิภัสสรา ส.เจริญพานิช'}, {'WA': '2016800P11006257 P11006231 THB       3000.00 ธัญรัศม์ พลมณี'}, {'WA': '2016800P11006257 P11006257 THB      30000.00 ชูกมล จิตระบอบ'}, {'WA': '2016800P11006257 P11010167 THB      30000.00 นภัสวรรณ เนตรภักดี'}, {'WA': '2016800P11006257 P11010175 THB      20000.00 ธนพร วงค์ชา'}, {'WA': '2016800P11006257 P11011614 THB      20000.00 ดรุณี กำลังศิลป์'}, {'WA': '2016800P11006257 P11012064 THB       3000.00 อรจิตร คำแก้ว'}, {'WA': '2016800P11006257 P11012207 THB       3000.00 โฉมเฉลา อินธิศักดิ์'}, {'WA': '2016800P11006257 P11012265 THB       3000.00 กัลยา คตธมาตย์'}, {'WA': '2016800P11006257 P11020470 THB       3000.00 วรณัน บุตนุ'}, {'WA': '2016800P11006257 P11027160 THB       3000.00 กุลประวีณ์ บุญเอนก'}, {'WA': '2016800P11006257 P11030245 THB       3000.00 สุพัตรา ถินอภัย'}, {'WA': '2016800P11006257 P11034787 THB       3000.00 ปัทมา  ถวิลการ'}, {'WA': '2016900P11005505 P11005505 THB      40000.00 ดาหวัน อรุณคีรีวัฒน์'}, {'WA': '2016900P11005505 P11010449 THB      10000.00 วรรณธณี ท้วมเทียบ'}, {'WA': '2016900P11005505 P11010596 THB      10000.00 วรรณี เมืองสมบัติ'}, {'WA': '2016900P11005505 P11034709 THB      10000.00 ฉัตรญาดา  กล้าปราบโจร'}, {'WA': '2016900P11010485 P11007861 THB      10000.00 ธนียา ขำพวง'}, {'WA': '2016900P11010485 P11009683 THB      10000.00 ชัชญา ปรีสุวรรณ'}, {'WA': '2016900P11010485 P11010485 THB      40000.00 จิรภรณ์ ผิวเผือก'}, {'WA': '2016900P11010485 P11010599 THB      10000.00 สนิท ผะอบเหล็ก'}, {'WA': '2016900P11010485 P11027345 THB      10000.00 นฤมล พรมดี'}, {'WA': '2017000P11024290 P11024290 THB      30000.00 เกษร ประสิทธ์พรพันธ์'}, {'WA': '2017100P9999999  P9999999  THB      30000.00 นายสมภพ กองทรัพย์,นางกมลวรรณ เขียวจ'}, {'WA': '2017300P1001216  P1001216  THB      30000.00 กฤษณา เงินสดย่อย'}, {'WA': '2017300P11000161 P11000161 THB      80000.00 ภานุวัฒน์ วัฒนา และ  นางธัญญาลักษณ์จันทร์สุวรรณ และ นายพรเทพ บัวทอง'}, {'WA': '2017300P11001905 P11001905 THB      30000.00 คนธีระ จินารักษ์ และ นางพาราดี วาสุและ  น.ส.สุภาวดี จีนอ่อน'}, {'WA': '2017300P11001954 P11001954 THB      20000.00 บัวจันทร์ อินตา  หรือ นางนงเยาว์มาร์จันทร์ หรือนายวิสูตร สังขมณี'}, {'WA': '2017300P11002164 P11002164 THB      30000.00 ทวินันท์ เนติพัฒน์ หรือ น.ส.พิชยาประภาพันธ์ หรือ นายอรรถพล ปิยภัณฑ์'}, {'WA': '2017300P11002549 P11002549 THB      20000.00 รัตนา  สุขสม หรือ นางกาญจนา อารยะธน'}, {'WA': '2017300P11002609 P11002609 THB      20000.00 ปรีญานุช  อุสาหะ หรือ นายณัฐพล  พรงนางกัญญาลักษ์ ปราดเปรียว'}, {'WA': '2017300P11002615 P11002615 THB      10000.00 นเรศร์  ธรรมศิริ และนางอารมย์รัฐธรรม หรือ น.ส.ชญารัศน์  สุดสนอง'}, {'WA': '2017300P11002642 P11002642 THB      50000.00 วิสูตร สังขมณี และนายนิวัฒน์ คำหมายและนายธีระ แสงชนินทร์รัตน์'}, {'WA': '2017300P11002653 P11002653 THB      50000.00 สนนึก เปรมบุญ น.ส.ปาณิสรา คำสมมาตร์น.ส.สุรางค์  จำปาทอง'}, {'WA': '2017300P11002679 P11002679 THB      20000.00 มั่น แจ่มเที่ยงตรง หรือ นายภีมพิชานันติแก้ว หรือ นายศุกล ธรรมศิริ'}, {'WA': '2017300P11002689 P11002689 THB      60000.00 อรษา พึ่งโพธิ และ พรสวรรค์ ส้วยเกร็นายจรัส กสิชีวิน'}, {'WA': '2017300P11004209 P11004209 THB      50000.00 ตริตราภรณ์ รักพรหม และ นางพัศรินทร์บางกุ้ง และ น.ส.วิลาวัลย์ บุญเจือ'}, {'WA': '2017300P11004214 P11004214 THB      50000.00 วรรณะ ศรีนวล และ น.ส.นวพรรณ  สีเข้มและ นางขนิษฐา คงสมเพ็ชร'}, {'WA': '2017300P11004711 P11004711 THB      30000.00 สมสิทธิ์ เหมกุล และ น.ส.นุสรา นาคทัและนายสมาน ลัคขณศิลป์'}, {'WA': '2017300P11005646 P11005646 THB      40000.00 จินตนา ธัญญการหรือน.ส.กฤชสร อนันต์สถิตย์หรือน.ส.ปริชาติ จันทร์ไข่'}, {'WA': '2017300P11007021 P11007021 THB      20000.00 ศักดิ์ดา มุ้งทอง หรือ นายวิสูตร สังหรือ นางสุนีย์ มณีแกม'}, {'WA': '2017300P11007415 P11007415 THB      60000.00 ชมพูนุท วรรณยิ่ง หรือ  นายวริษฐ์สรรพวิสุทธิ์ หรือ นางมาลัยพร สุขสมบ'}, {'WA': '2017300P11007645 P11007645 THB      40000.00 สุธีรัตน์ หลอมทอง และ นางสาวิตรี ส่และ นางศศิญดา ฉัตรวิไล'}, {'WA': '2017300P11008789 P11008789 THB      50000.00 กันยารัตน์ แสงเดช และ นายวิสิทธิ์ ทและ นายธวัช  เต็มภาชนะ'}, {'WA': '2017300P11011190 P11011190 THB      15000.00 สิงหล เสือทอง หรือ น.ส.รุ่งนภาสายรื่น หรือ น.ส.มัญชุสา สุวรรณคง'}, {'WA': '2017300P11013184 P11013184 THB      60000.00 ธีรพล สุทธิพลสมบูรณ์ หรือ น.ส.กัญกรคงวงศ์วาน หรือ น.ส.อัญญฎา  ภูท้วม'}, {'WA': '2017300P11013655 P11013655 THB      50000.00 ยศพงษ์  เทพโยธิน และ น.ส.อุไรวรรณคำอุดม และ น.ส.วรรณพร'}, {'WA': '2017300P11014198 P11014198 THB      20000.00 พีระ สิริรุ่งอมร หรือ นายอรรคเดชศิริรักษ์ หรือ นายศุกล ธรรมศิริ'}, {'WA': '2017300P11014226 P11014226 THB      50000.00 นภัสสร โคตพงษ์ หรือ น.ส.ศศิวิมล พิรหรือ นางจุฑามาศ อยู่รอย'}, {'WA': '2017300P12034591 P12034591 THB      50000.00 ธีระวัฒน์ แสงงาม และ น.ส.อุบลบุตนุ และ น.ส.ดารุณี สานา'}, {'WA': '2017300P12039734 P12039734 THB      40000.00 ธวัชชัย น้อยจันทร์ หรือ น.ส.จิราวรรพลพิทักษ์ หรือ น.ส.วรรณวิภา มีจันทร'}, {'WA': '2017300P12048652 P12048652 THB      50000.00 เดือนใส แดงประเสริฐ หรือนายสุนทรวิเศษพันธ์ หรือน.ส.สุปราณี เพียนไธส'}, {'WA': '2017400P11028004 P11028004 THB     100000.00 กิตติกรรัตนาวลี วงศ์ราชนุช'}, {'WA': '2017500P0750220  P0750220  THB      40000.00 สำนักงานขนส่ง-วังน้อย'}, {'WA': '2017500P0750320  P0750320  THB     735000.00 สำนักงานขนส่ง-บางบาล'}, {'WA': '2017500P0751210  P0751210  THB      20000.00 คลัง MT 51,56'}, {'WA': '2017500P0751210  P11024068 THB      50000.00 วรลักษณ์ เมฆพลัด'}, {'WA': '2017500P0751210  P11024219 THB      50000.00 สหรัฐ บุญปลื้ม'}, {'WA': '2017500P0751330  P0751330  THB      10000.00 คลัง-TTDC'}, {'WA': '2017500P0753000  P0753000  THB      20000.00 สำนักงาน-DC-กรุงเทพเหนือ(มหาทิพย์)'}, {'WA': '2017500P0753400  P0753400  THB      10000.00 RDC-นครราชสีมา'}, {'WA': '2017500P0754010  P0754010  THB      18000.00 ศูนย์กระจายสินค้า-RDC.ลำปาง'}, {'WA': '2017500P0754700  P0754700  THB      90000.00 RDC-สุราษฎร์ธานี'}, {'WA': '2017500P0755020  P0755020  THB      10000.00 สำนักงานขนส่ง-กาญจนสิงขร'}, {'WA': '2017500P0755030  P0755030  THB       5000.00 สำนักงานขนส่ง-หลักชัย'}, {'WA': '2017500P0755040  P0755040  THB      20000.00 ศูนย์ย่อยเฟื่องฟู-ปราจีนบุรี'}, {'WA': '2017500P0755050  P0755050  THB      10000.00 ศูนย์ขนส่งย่อย อธิมาตร'}, {'WA': '2017500P0755060  P0755060  THB      10000.00 ศูนย์ขนส่งย่อย เอส เอส การสุรา'}, {'WA': '2017500P0755070  P0755070  THB      10000.00 ศูนย์ขนส่งย่อย แก่นขวัญ'}, {'WA': '2017500P0755080  P0755080  THB      10000.00 ศูนย์ขนส่งย่อย เทพอรุโณทัย'}, {'WA': '2017500P0755090  P0755090  THB      10000.00 ศูนย์ย่อย มงคลสมัย'}, {'WA': '2017500P0755100  P0755100  THB      20000.00 สำนักงานขนส่ง-บางยี่ขัน'}, {'WA': '2017500P0755130  P0755130  THB      20000.00 สำนักงานขนส่ง-กระทิงแดง'}, {'WA': '2017500P0755160  P0755160  THB      10000.00 สำนักงานขนส่ง-แสงโสม'}, {'WA': '2017500P0756010  P0756010  THB      10000.00 คลังจังหวัด ปราจีนบุรี'}, {'WA': '2017500P0756031  P0756031  THB      50000.00 คลังจังหวัด ชุมพร'}, {'WA': '2017500P0756033  P0756033  THB      30000.00 คลังจังหวัด ภูเก็ต'}, {'WA': '2017500P0756035  P0756035  THB      30000.00 คลังจังหวัด กระบี่'}, {'WA': '2017500P0756036  P0756036  THB      60000.00 คลังจังหวัด นครศรีธรรมราช'}, {'WA': '2017500P0756037  P0756037  THB      40000.00 คลังจังหวัด สงขลา'}, {'WA': '2017500P0756038  P0756038  THB      25000.00 คลังจังหวัด ตรัง'}, {'WA': '2017500P0756039  P0756039  THB      10000.00 คลังจังหวัด พัทลุง'}, {'WA': '2017500P0756040  P0756040  THB      20000.00 คลังจังหวัดจันทบุรี'}, {'WA': '2017500P0756050  P0756050  THB       5000.00 คลังจังหวัดสระแก้ว'}, {'WA': '2017500P0756060  P0756060  THB      18000.00 คลังจังหวัดชลบุรี'}, {'WA': '2017500P0756070  P0756070  THB       5000.00 คลังจังหวัดฉะเชิงเทรา'}, {'WA': '2017500P0756080  P0756080  THB       8000.00 คลังจังหวัดระยอง'}, {'WA': '2017500P0756090  P0756090  THB      10000.00 คลังจังหวัด บุรีรัมย์'}, {'WA': '2017500P0756100  P0756100  THB      10000.00 คลังจังหวัด นครราชสีมา'}, {'WA': '2017500P0756110  P0756110  THB      10000.00 คลังจังหวัด สีคิ้ว'}, {'WA': '2017500P0756130  P0756130  THB      10000.00 คลังจังหวัด สุรินทร์'}, {'WA': '2017500P0756140  P0756140  THB      15000.00 คลังจังหวัด อุบลราชธานี'}, {'WA': '2017500P0756150  P0756150  THB      10000.00 คลังจังหวัด ศรีสะเกษ'}, {'WA': '2017500P0756160  P0756160  THB      10000.00 คลังจังหวัด ยโสธร'}, {'WA': '2017500P0756170  P0756170  THB      10000.00 คลังจังหวัด ร้อยเอ็ด'}, {'WA': '2017500P0756180  P0756180  THB      10000.00 คลังจังหวัด อำนาจเจริญ'}, {'WA': '2017500P0756190  P0756190  THB      20000.00 คลังจังหวัด ขอนแก่น'}, {'WA': '2017500P0756200  P0756200  THB      10000.00 คลังจังหวัด ชัยภูมิ'}, {'WA': '2017500P0756210  P0756210  THB      10000.00 คลังจังหวัด กาฬสินธุ์'}, {'WA': '2017500P0756220  P0756220  THB      10000.00 คลังจังหวัด มหาสารคาม'}, {'WA': '2017500P0756230  P0756230  THB      10000.00 คลังจังหวัด อุดรธานี'}, {'WA': '2017500P0756240  P0756240  THB      10000.00 คลังจังหวัด หนองคาย'}, {'WA': '2017500P0756250  P0756250  THB      10000.00 คลังจังหวัด สกลนคร'}, {'WA': '2017500P0756260  P0756260  THB      10000.00 คลังจังหวัด นครพนม'}, {'WA': '2017500P0756270  P0756270  THB      10000.00 คลังจังหวัด มุกดาหาร'}, {'WA': '2017500P0756280  P0756280  THB      10000.00 คลังจังหวัด เลย'}, {'WA': '2017500P0756440  P0756440  THB      10000.00 คลังจังหวัด นครปฐม'}, {'WA': '2017500P0756450  P0756450  THB       5000.00 คลังจังหวัด สุรพรรณบุรี'}, {'WA': '2017500P0756460  P0756460  THB       5000.00 คลังจังหวัด กาญจนบุรี'}, {'WA': '2017500P0756470  P0756470  THB      10000.00 คลังจังหวัด ราชบุรี'}, {'WA': '2017500P0756500  P0756500  THB       5000.00 คลังจังหวัด เพชรบุรี'}, {'WA': '2017500P0756510  P0756510  THB      10000.00 คลังจังหวัด ประจวบคีรีขันท์'}, {'WA': '2017500P0756520  P0756520  THB      20000.00 คลังจังหวัด นครสวรรค์'}, {'WA': '2017500P0756530  P0756530  THB       8000.00 คลังจังหวัด พิจิตร'}, {'WA': '2017500P0756540  P0756540  THB       8000.00 คลังจังหวัด เพชรบูรณ์'}, {'WA': '2017500P0756550  P0756550  THB       8000.00 คลังจังหวัด กำแพงเพชร'}, {'WA': '2017500P0756560  P0756560  THB       8000.00 คลังจังหวัด อยุธยา'}, {'WA': '2017500P0756590  P0756590  THB      20000.00 คลังจังหวัด สระบุรี'}, {'WA': '2017500P0756600  P0756600  THB      15000.00 คลังจังหวัด ลพบุรี'}, {'WA': '2017500P0756610  P0756610  THB      15000.00 คลังจังหวัด ชัยนาท'}, {'WA': '2017500P0756630  P0756630  THB      13000.00 คลังจังหวัด เชียงใหม่1'}, {'WA': '2017500P0756640  P0756640  THB       8000.00 คลังจังหวัด เชียงใหม่-ฝาง'}, {'WA': '2017500P0756650  P0756650  THB      10000.00 คลังจังหวัด เชียงราย'}, {'WA': '2017500P0756690  P0756690  THB      16000.00 คลังจังหวัด เชียงใหม่-แม่แตง'}, {'WA': '2017500P0756700  P0756700  THB       8000.00 คลังจังหวัด พิษณุโลก'}, {'WA': '2017500P0756710  P0756710  THB       8000.00 คลังจังหวัด สุโขทัย'}, {'WA': '2017500P0756720  P0756720  THB       4000.00 คลังจังหวัด ตาก'}, {'WA': '2017500P0756730  P0756730  THB       8000.00 คลังจังหวัด อุตรดิตถ์'}, {'WA': '2017500P0756740  P0756740  THB       8000.00 คลังจังหวัด แพร่'}, {'WA': '2017500P0756760  P0756760  THB      10000.00 คลังสมุทรปราการ'}, {'WA': '2017500P0756770  P0756770  THB      20000.00 คลังบางขัน(ปทุมธานี)'}, {'WA': '2017500P0756800  P0756800  THB      10000.00 คลังหทัยราษฏร์(มีนบุรี)'}, {'WA': '2017500P0756810  P0756810  THB      20000.00 คลังพรีเม็ค(บางนา)'}, {'WA': '2017500P0756860  P0756860  THB       8000.00 คลังจังหวัด เชียงใหม่-ฮอด'}, {'WA': '2017500P0756890  P0756890  THB       4000.00 คลังจังหวัดตาก(แม่สอด)'}, {'WA': '2017500P1001742  P1001742  THB      50000.00 วรัญญา เงินสดย่อย'}, {'WA': '2017500P11002830 E11004927 THB          0.00 ชื่นจิต เชื้อวงษ์'}, {'WA': '2017500P11010159 P11010159 THB      35000.00 กรณ์ชนก ฟักทองอยู่'}, {'WA': '2017500P11016970 P11016970 THB      80000.00 อันธิกา  กิจการ'}, {'WA': '2017500P11018565 P11018565 THB     300000.00 ปิยธิดา  รัตนพันธุ์เศรษฐ'}, {'WA': '2017500P11018993 P11018993 THB     100000.00 จิตรา วรสัตย์'}, {'WA': '2017500P11033138 P11033138 THB      50000.00 แสงโสม Petty Cash'}, {'WA': '2017500P11033138 P11033139 THB      30000.00 พรทิพย์ ปิยะอรุณ Test'}, {'WA': '2017500P11033139 P11033139 THB         50.26 พรทิพย์ ปิยะอรุณ Test'}, {'WA': '2017500P70017550 P70017550 THB      20000.00 ภาวินี สุวรรณโยธี'}, {'WA': '2017500P755170   P755170   THB      10000.00 สำนักงานขนส่งย่อย-ภาค 1'}, {'WA': '2017500P756832   P756832   THB      20000.00 คลังบุคคโล'}, {'WA': '20175F0P11032391 P11032391 THB      15000.00 ทดสอบ havifood'}, {'WA': '20175F0P70001112 P70001112 THB      45000.00 Test haviifood'}, {'WA': '20175L0P11032391 P11032391 THB      15000.00 ทดสอบ haviflo'}, {'WA': '20175L0P11032391 P70005915 THB       5000.00 สุวรรณา เจริญทรัพย์'}, {'WA': '20175L0P70001111 P70001111 THB      40000.00 Test havilogis'}, {'WA': '20175L0P70001111 P70001113 THB      10000.00 Test 1111'}, {'WA': '20175L0P70003867 P70005915 THB      40000.00 สุวรรณา เจริญทรัพย์'}, {'WA': '2017600P11002371 P11002371 THB      30000.00 นุชจรินทร์ นิ้มวัฒนา'}, {'WA': '2017600P11005402 P11005402 THB      10000.00 วัลวิภา  หร่ายเกษม'}, {'WA': '2017600P76102000 P76102000 THB      15000.00 ณัฐพงศ์ มะอะอุ'}, {'WA': '2017700P11001794 P11001794 THB      20000.00 องอาจ  ไสวชัยศรี'}, {'WA': '2017700P11004814 P11004814 THB      20000.00 ชัยวัฒน์ จันทิมา'}, {'WA': '2017700P11020023 P11020023 THB      20000.00 สุรชัย เนียมปาน'}, {'WA': '2017700P11022675 P11022675 THB      20000.00 พนิดา ใจกระจ่าง'}, {'WA': '2017700P11024617 P11024617 THB      20000.00 พัชรา มาโคตร'}, {'WA': '2017700P70039808 P70039808 THB      20000.00 นิรดา ธีรวงษ์ไพบูลย์'}, {'WA': '2017800P11005608 P11005284 THB      10000.00 คณึงนิตย์ พันละเกตุ'}, {'WA': '2017800P11005608 P11005608 THB      50000.00 เปรมจิตต์ ธรรมไพบูลย์ เปรมจิตต์ ธรรเปรมจิตต์ ธรรมไพบูลย์ เปรมจิตต์ ธรรเปรมจิตต์ ธรรมไพบูลย์ เปรมจิตต์ ธรรเปรมจิตต์ ธรรมไพบูลย์ เปรมจิตต์ ธรร'}, {'WA': '2017800P11005608 P11005921 THB       8000.00 กวิตา ประดงจงเนตร'}, {'WA': '2017800P11005608 P11006188 THB       3000.00 สุธิน ทวนประเสริฐ'}, {'WA': '2017800P11005608 P12039112 THB      15000.00 ปรียา เรือนทองคำ'}, {'WA': '2017800P11005608 P21034579 THB       3000.00 ปถมาพร พาราพิบูลย์'}, {'WA': '2017800P11005608 P70002909 THB       3000.00 ติรยา สรรพอุดม'}, {'WA': '2017800P11013655 P11013655              0.00 ยศพงษ์  เทพโยธิน และ น.ส.อุไรวรรณคำอุดม และ น.ส.วรรณพร'}, {'WA': '2017900P11005505 P11005505 THB      10000.00 ดาหวัน อรุณคีรีวัฒน์'}, {'WA': '2017900P11005505 P11010449 THB       5000.00 วรรณธณี ท้วมเทียบ'}, {'WA': '2017900P11005505 P11010596 THB       5000.00 วรรณี เมืองสมบัติ'}, {'WA': '2017900P11005505 P11034709 THB       3000.00 ฉัตรญาดา  กล้าปราบโจร'}, {'WA': '2017900P11006184 P11006184 THB      40000.00 อรวีร์ ด่านกุล'}, {'WA': '2017900P11006184 P11007861 THB       5000.00 ธนียา ขำพวง'}, {'WA': '2017900P11006184 P11009683 THB       5000.00 ชัชญา ปรีสุวรรณ'}, {'WA': '2017900P11006184 P11010599 THB       5000.00 สนิท ผะอบเหล็ก'}, {'WA': '2017900P11006184 P11027345 THB       3000.00 นฤมล พรมดี'}, {'WA': '2018600P11005402 P11005402 THB      30000.00 วัลวิภา  หร่ายเกษม'}, {'WA': '2018600P11023092 P11023092 THB      30000.00 กมล เศวตทวี'}, {'WA': '2018700P11010169 P11010167 THB      40000.00 นภัสวรรณ เนตรภักดี'}, {'WA': '2018700P11010169 P11010169 THB      50000.00 พรฑิตา เวียงหก'}, {'WA': '2018700P11010169 P11010175 THB      40000.00 ธนพร วงค์ชา'}, {'WA': '2018700P11010169 P11011614 THB      40000.00 ดรุณี กำลังศิลป์'}, {'WA': '2018900P70027834 P70027834 THB     150000.00 ปนัดดา เอี่ยมสะอาด'}, {'WA': '2019000P11026768 P11026768 THB      30000.00 พันธ์ทิพย์ ชัยชมภู'}, {'WA': '2019000P11030318 P11030318 THB      50000.00 ชวัลรัสย์ คำทะ'}, {'WA': '20190B0P1001742  P1001742  THB      40000.00 วรัญญา เงินสดย่อย'}, {'WA': '20190B0P11027937 P11027937 THB      10000.00 วชีรา สุขบรรเทิง'}, {'WA': '20190B0P11027937 P11030280 THB      10000.00 พิมลวรรณ มหาชัย'}, {'WA': '20190B0P11030280 P11030280 THB      30000.00 พิมลวรรณ มหาชัย'}, {'WA': '20190B0P11030318 P11030318 THB      50000.00 ชวัลรัสย์ คำทะ'}, {'WA': '2019100P01001174 P01001174 THB      50000.00 ปาณิศา ปรัชญาดำรงพล'}, {'WA': '2019100P70092728 P70092728 THB      50000.00 กิตติอาภา อ่างบุญพงษ์'}, {'WA': '2019300P01001174 P01001174 THB      50000.00 ปาณิศา ปรัชญาดำรงพล'}]

In [ ]:
FIELD = [{'FIELDNAME': 'MANDT', 'OFFSET': '000000', 'LENGTH': '000003', 'TYPE': 'C', 'FIELDTEXT': 'Client'}, {'FIELDNAME': 'BUKRS', 'OFFSET': '000003', 'LENGTH': '000004', 'TYPE': 'C', 'FIELDTEXT': 'Company Code'}, {'FIELDNAME': 'LIFNR1', 'OFFSET': '000007', 'LENGTH': '000010', 'TYPE': 'C', 'FIELDTEXT': 'Account Number of Vendor or Creditor'}, {'FIELDNAME': 'LIFNR2', 'OFFSET': '000017', 'LENGTH': '000010', 'TYPE': 'C', 'FIELDTEXT': 'Account Number of Vendor or Creditor'}, {'FIELDNAME': 'CURRENCY', 'OFFSET': '000027', 'LENGTH': '000005', 'TYPE': 'C', 'FIELDTEXT': 'Currency Key'}, {'FIELDNAME': 'AMOUNT', 'OFFSET': '000032', 'LENGTH': '000013', 'TYPE': 'P', 'FIELDTEXT': 'Amount in Document Currency'}, {'FIELDNAME': 'VNAME', 'OFFSET': '000045', 'LENGTH': '000140', 'TYPE': 'C', 'FIELDTEXT': 'Vendor Name'}]

In [ ]:
FIELDTEXT = [f['FIELDTEXT'] for f in FIELD ]
print(FIELDTEXT)

In [ ]:
columns = [f['FIELDNAME'] for f in FIELD ]
print(columns)

In [ ]:
formatted_data = [list(map(str.strip, (row['WA'] or '').split())) for row in DATA]

In [ ]:
DATA

In [ ]:
formatted_data[0]

In [ ]:
pandas.DataFrame(data=None, index=None, columns=None, 

In [ ]:
DATA[0]['WA']

In [ ]:
DATA[1]['WA']

In [ ]:
DATA[1]['WA'].split(" ")

In [ ]:
 np.isnan(  np.average( history.history['kl_loss'] )  )

In [ ]:
#nan problem
#partdate = '2022-05'
#normal 
#partdate = '2011-12'

In [ ]:
#DATA

In [ ]:
# partdate = '2011-12'
# partdate = '2022-05'
# with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
#     DATA = f[partdate][:]
#     X = DATA[:, :, :3]  # เลือกข้อมูลแถวแรกถึงแถวที่ 3 สำหรับ X
#     Y = DATA[:, :, 3:]  # เลือกข้อมูลแถวที่ 3 เป็นต้นไปสำหรับ Y

In [ ]:
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]
F=[1,2,3,4,5]
for input_name, decoder_output in zip(colList, Y[:3]):
    print(input_name)
    print(decoder_output)
    print("==============")

In [ ]:
tf.keras.losses.MSE(
    Y[2], Y[3]
)

In [ ]:
A = tf.constant(Y[2], dtype=tf.float32)
B = tf.constant(Y[3], dtype=tf.float32)

In [ ]:
 tf.reduce_mean( tf.square(A[:,0]-B[:,0]) )

In [ ]:
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]

In [ ]:
tf.reduce_sum(A_first_feature, axis=0, keepdims=True)

In [ ]:
 tf.reduce_sum()

In [ ]:
X[2]

In [ ]:
for i in range ( len( np.transpose( Y[0] ) ) ):
    col = np.transpose( Y[i] ) 
    for c in range(len(col)):
        a = np.transpose( Y[i] )[c]
        if np.sum( a ) == 0 :
            print(f"0 - i:{i},c:{c}")
        if np.isnan( np.sum( a ) ) :
            print(f"nan - i:{i},c:{c}")

In [ ]:
for i in range ( len( np.transpose( X[0] ) ) ):
    col = np.transpose( X[i] ) 
    for c in range(len(col)):
        a = np.transpose( X[i] )[c]
        if np.sum( a ) == 0 :
            print(f"0 - i:{i},c:{c}")
        if np.isnan( np.sum( a ) ) :
            print(f"nan - i:{i},c:{c}")

In [ ]:
Y[0] 

In [ ]:
np.transpose( Y[0] )

In [ ]:
np.transpose( Y[2] )[5]

In [ ]:
X[2]

In [ ]:
Y[2]

In [ ]:
X[2][8:12]

In [ ]:
import tensorflow as tf

# Tensor with a nan value
tensor_with_nan = tf.constant([[0.0, 0, 0], [0, 0, 0]])
# Perform reduce_mean
mean_with_nan = tf.reduce_mean(tensor_with_nan)
print("Mean of tensor with nan:", mean_with_nan.numpy())


In [ ]:
tensor_with_nan.numpy()

In [ ]:
history.history

In [ ]:
Y[2][-10]

In [ ]:
for i in range( len(X) ): 
    if np.sum( X[i] ) == 0 :
        print(   np.sum( X[i] ) )

In [ ]:
len(X)

In [ ]:
for i in range( len(Y) ): 
    if np.sum( Y[i] ) == 0 :
        print(   np.sum( Y[i] ) )

In [ ]:
np.average(  history.history['kl_loss'] )

In [ ]:
partdate

In [ ]:
Y[5]

In [ ]:
X

In [ ]:
f = h5py.File(H5_PATH+'2012-05'+".h5", 'r') 

In [ ]:
f.keys()

In [ ]:
import numpy as np

def create_adjacency_matrix(options, threshold=25):
    """
    สร้าง adjacency matrix สำหรับ Options Chain โดยใช้ความใกล้เคียงของ strike price และ DTE
    """
    num_options = len(options)
    adjacency_matrix = np.zeros((num_options, num_options))

    for i in range(num_options):
        for j in range(i, num_options):
            # พิจารณาเชื่อมโยงระหว่าง options หาก strike price ต่างกันไม่เกิน threshold และ DTE เท่ากัน
            if abs(options[i]['strike_price'] - options[j]['strike_price']) <= threshold and options[i]['dte'] == options[j]['dte']:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    
    return adjacency_matrix

options = [
    {'strike_price': 100, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 115, 'dte': 30},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 0, 'dte': 0},  # เปลี่ยนค่าเป็น 0
]

options = [
    {'strike_price': 0, 'dte': 0},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 150, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 100, 'dte': 30},  # เปลี่ยนค่าเป็น 0
]


# ปรับ threshold
threshold = 5

adjacency_matrix = create_adjacency_matrix(options, threshold)
print(adjacency_matrix)

In [ ]:
import numpy as np
A = np.array(
    [[1,2,3],
    [4,5,6],
    [7,8,9]]
    
)

In [ ]:
A[:,:,0]

In [ ]:
import numpy as np

# Generate a random numpy array with shape (2, 20, 6)
random_array = np.random.rand(2, 20, 6)

print(random_array)

In [ ]:
random_array = np.random.rand(2, 5, 3)

In [ ]:
random_array

In [ ]:
random_array[:, :, -1]

In [ ]:
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]

In [ ]:
colList.index("c_ask")